# Hyperparameter tuning for fine-tuned Chronos-2 models

Questo notebook avvia l'iper-tuning partendo da checkpoint **già finetunati**.
I checkpoint si trovano nella cartella `outputs/chronos2_sft` (generale e per categorie).


In [ ]:
import json
import sys
from pathlib import Path

# Ensure project root is on sys.path when running from notebooks/
ROOT = Path('..').resolve()
if str(ROOT) not in sys.path:
    sys.path.insert(0, str(ROOT))
if str(ROOT / 'model') not in sys.path:
    sys.path.insert(0, str(ROOT / 'model'))

import torch

from chronos import Chronos2Pipeline
from core.data import GICS_LEVEL_1, create_multivariate_windows, prepare_data_for_chronos
from core.eval import evaluate_model_on_test
from tiingo_data.download_data import get_daily_returns_data_cached
from utils import create_train_val_split, get_device

# Example: create a JSON mapping for category models
category_models = {
    'Information Technology': 'outputs/chronos2_sft/categories/Information_Technology/finetuned-ckpt',
    'Health Care': 'outputs/chronos2_sft/categories/Health_Care/finetuned-ckpt',
}
Path('category_models.json').write_text(json.dumps(category_models, indent=2), encoding='utf-8')
print('Wrote category_models.json')

# Config
context_length = 200
prediction_length = 1
stride = 50
min_past = 200
test_size = 1200
val_ratio = 0.2
n_samples = 100
seed = 42

grid = {
    'learning_rate': [1e-6, 5e-6, 1e-5],
    'num_steps': [500, 1000],
    'batch_size': [16, 32],
}

# Data prep
torch.manual_seed(seed)
df_all = get_daily_returns_data_cached()
df_train_clean, _ = prepare_data_for_chronos(df_all, test_size=test_size)
df_train_split, df_val_split = create_train_val_split(df_train_clean, val_ratio=val_ratio)

def build_inputs(df_train, df_val):
    train_inputs = create_multivariate_windows(
        df_train,
        context_length=context_length,
        prediction_length=prediction_length,
        stride=stride,
    )
    val_inputs = create_multivariate_windows(
        df_val,
        context_length=context_length,
        prediction_length=prediction_length,
        stride=stride,
    )
    return train_inputs, val_inputs

def tune_model(model_path, train_inputs, val_inputs, df_val):
    results = []
    device = get_device()
    for lr in grid['learning_rate']:
        for steps in grid['num_steps']:
            for batch_size in grid['batch_size']:
                print(f'Tuning: lr={lr}, steps={steps}, batch={batch_size}')
                pipeline = Chronos2Pipeline.from_pretrained(
                    model_path,
                    device_map=device,
                    dtype=torch.float32,
                )
                finetuned = pipeline.fit(
                    inputs=train_inputs,
                    validation_inputs=val_inputs,
                    prediction_length=prediction_length,
                    context_length=context_length,
                    min_past=min_past,
                    num_steps=steps,
                    batch_size=batch_size,
                    learning_rate=lr,
                    output_dir=ROOT / 'outputs' / 'hyperparameter_tuning',
                )
                metrics = evaluate_model_on_test(
                    pipeline=finetuned,
                    df_test=df_val,
                    context_length=context_length,
                    n_samples=n_samples,
                )
                results.append({
                    'config': {'learning_rate': lr, 'num_steps': steps, 'batch_size': batch_size},
                    'metrics': metrics,
                })
    return results

def top_k(results, k=3):
    return sorted(results, key=lambda x: x['metrics']['mean_quantile_loss'])[:k]

# General model tuning
general_model_path = 'outputs/chronos2_sft/finetuned-ckpt'
train_inputs, val_inputs = build_inputs(df_train_split, df_val_split)
general_results = tune_model(general_model_path, train_inputs, val_inputs, df_val_split)
top_general = top_k(general_results, k=3)

# Category models tuning
top_categories = {}
for category, tickers in GICS_LEVEL_1.items():
    model_path = category_models.get(category)
    if not model_path:
        continue
    available = [t for t in tickers if t in df_train_split.columns]
    if not available:
        continue
    train_df = df_train_split[available]
    val_df = df_val_split[available]
    train_inputs, val_inputs = build_inputs(train_df, val_df)
    results = tune_model(model_path, train_inputs, val_inputs, val_df)
    top_categories[category] = top_k(results, k=3)

summary = {
    'general': top_general,
    'categories': top_categories,
}
Path('top_configs.json').write_text(json.dumps(summary, indent=2), encoding='utf-8')
print('Saved top_configs.json')
